In [1]:
import pandas as pd
import math
import random
# from matplotlib.cm import binary
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
# from sklearn.datasets import fetch_openml
# from sklearn.ensemble  import RandomForestClassifier,RandomForestRegressor
# from sklearn.svm import SVC
# from sklearn import svm
# from sklearn.neural_network import MLPClassifier
# from sklearn.linear_model import SGDClassifier, LogisticRegression,LinearRegression
# from sklearn.metrics import confusion_matrix,classification_report,precision_score,recall_score,f1_score,precision_recall_curve
# from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedShuffleSplit,cross_val_predict
# from sklearn.metrics import accuracy_score,mean_squared_error
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.tree import DecisionTreeClassifier
from pandas.plotting import scatter_matrix
from statsmodels.graphics.mosaicplot import mosaic
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.tree import DecisionTreeRegressor
from sklearn.cluster import KMeans
%matplotlib inline

In [2]:
df = pd.read_csv('retail.csv')
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12-01-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12-01-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12-01-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12-01-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12-01-2010 08:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
995,536520,22469,HEART OF WICKER SMALL,1,12-01-2010 12:43,1.65,14729.0,United Kingdom
996,536520,22100,SKULLS SQUARE TISSUE BOX,1,12-01-2010 12:43,1.25,14729.0,United Kingdom
997,536520,22096,PINK PAISLEY SQUARE TISSUE BOX,1,12-01-2010 12:43,1.25,14729.0,United Kingdom
998,536520,22583,PACK OF 6 HANDBAG GIFT BOXES,1,12-01-2010 12:43,2.55,14729.0,United Kingdom


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    1000 non-null   object 
 1   StockCode    1000 non-null   object 
 2   Description  999 non-null    object 
 3   Quantity     1000 non-null   int64  
 4   InvoiceDate  1000 non-null   object 
 5   UnitPrice    1000 non-null   float64
 6   CustomerID   999 non-null    float64
 7   Country      1000 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 62.6+ KB


In [4]:
df['Description'] = df['Description'].str.strip() 

In [5]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    1
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     1
Country        0
dtype: int64

In [6]:
df.dropna(subset=['Description','CustomerID'],axis=0,inplace=True)

In [7]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [8]:
basket = (df[df['Country']=='United Kingdom'].groupby(['InvoiceNo','Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo'))
basket

Description,10 COLOUR SPACEBOY PEN,12 DAISY PEGS IN WOOD BOX,12 MESSAGE CARDS WITH ENVELOPES,12 PENCILS TALL TUBE SKULLS,3 PIECE SPACEBOY COOKIE CUTTER SET,3 STRIPEY MICE FELTCRAFT,3 TIER CAKE TIN GREEN AND CREAM,3 TIER CAKE TIN RED AND CREAM,36 FOIL HEART CAKE CASES,36 FOIL STAR CAKE CASES,...,WOODEN FRAME ANTIQUE WHITE,WOODEN HEART CHRISTMAS SCANDINAVIAN,WOODEN OWLS LIGHT GARLAND,WOODEN PICTURE FRAME WHITE FINISH,WRAP COWBOYS,YELLOW BREAKFAST CUP AND SAUCER,YELLOW COAT RACK PARIS FASHION,YOU'RE CONFUSING ME METAL SIGN,ZINC METAL HEART DECORATION,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536520,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C536379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C536383,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
def encode_units(x):
    if x<=0:
        return 0
    if x>=1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets

Description,10 COLOUR SPACEBOY PEN,12 DAISY PEGS IN WOOD BOX,12 MESSAGE CARDS WITH ENVELOPES,12 PENCILS TALL TUBE SKULLS,3 PIECE SPACEBOY COOKIE CUTTER SET,3 STRIPEY MICE FELTCRAFT,3 TIER CAKE TIN GREEN AND CREAM,3 TIER CAKE TIN RED AND CREAM,36 FOIL HEART CAKE CASES,36 FOIL STAR CAKE CASES,...,WOODEN FRAME ANTIQUE WHITE,WOODEN HEART CHRISTMAS SCANDINAVIAN,WOODEN OWLS LIGHT GARLAND,WOODEN PICTURE FRAME WHITE FINISH,WRAP COWBOYS,YELLOW BREAKFAST CUP AND SAUCER,YELLOW COAT RACK PARIS FASHION,YOU'RE CONFUSING ME METAL SIGN,ZINC METAL HEART DECORATION,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536520,0,0,1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
C536379,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C536383,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
frequent_items = apriori(basket_sets,min_support = 0.07,use_colnames=True)
rules = association_rules(frequent_items,metric='confidence',min_threshold=1)
rules

C:\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(GLASS STAR FROSTED T-LIGHT HOLDER),(CREAM CUPID HEARTS COAT HANGER),0.080645,0.080645,0.080645,1.0,12.400000,0.074142,inf
1,(CREAM CUPID HEARTS COAT HANGER),(GLASS STAR FROSTED T-LIGHT HOLDER),0.080645,0.080645,0.080645,1.0,12.400000,0.074142,inf
2,(CREAM CUPID HEARTS COAT HANGER),(KNITTED UNION FLAG HOT WATER BOTTLE),0.080645,0.096774,0.080645,1.0,10.333333,0.072841,inf
3,(CREAM CUPID HEARTS COAT HANGER),(RED WOOLLY HOTTIE WHITE HEART.),0.080645,0.145161,0.080645,1.0,6.888889,0.068939,inf
4,(CREAM CUPID HEARTS COAT HANGER),(SET 7 BABUSHKA NESTING BOXES),0.080645,0.096774,0.080645,1.0,10.333333,0.072841,inf
...,...,...,...,...,...,...,...,...,...
1647,"(WHITE METAL LANTERN, SET 7 BABUSHKA NESTING B...","(RED WOOLLY HOTTIE WHITE HEART., CREAM CUPID H...",0.080645,0.080645,0.080645,1.0,12.400000,0.074142,inf
1648,"(SET 7 BABUSHKA NESTING BOXES, KNITTED UNION F...","(RED WOOLLY HOTTIE WHITE HEART., CREAM CUPID H...",0.080645,0.080645,0.080645,1.0,12.400000,0.074142,inf
1649,(CREAM CUPID HEARTS COAT HANGER),"(RED WOOLLY HOTTIE WHITE HEART., WHITE HANGING...",0.080645,0.080645,0.080645,1.0,12.400000,0.074142,inf
1650,(GLASS STAR FROSTED T-LIGHT HOLDER),"(RED WOOLLY HOTTIE WHITE HEART., CREAM CUPID H...",0.080645,0.080645,0.080645,1.0,12.400000,0.074142,inf


In [31]:
rules = rules[(rules['lift']>=12.4) & (rules['support']>=0.08)]
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(GLASS STAR FROSTED T-LIGHT HOLDER),(CREAM CUPID HEARTS COAT HANGER),0.080645,0.080645,0.080645,1.0,12.4,0.074142,inf
1,(CREAM CUPID HEARTS COAT HANGER),(GLASS STAR FROSTED T-LIGHT HOLDER),0.080645,0.080645,0.080645,1.0,12.4,0.074142,inf
6,(WHITE METAL LANTERN),(CREAM CUPID HEARTS COAT HANGER),0.080645,0.080645,0.080645,1.0,12.4,0.074142,inf
7,(CREAM CUPID HEARTS COAT HANGER),(WHITE METAL LANTERN),0.080645,0.080645,0.080645,1.0,12.4,0.074142,inf
12,(WHITE METAL LANTERN),(GLASS STAR FROSTED T-LIGHT HOLDER),0.080645,0.080645,0.080645,1.0,12.4,0.074142,inf
...,...,...,...,...,...,...,...,...,...
1647,"(WHITE METAL LANTERN, SET 7 BABUSHKA NESTING B...","(RED WOOLLY HOTTIE WHITE HEART., CREAM CUPID H...",0.080645,0.080645,0.080645,1.0,12.4,0.074142,inf
1648,"(SET 7 BABUSHKA NESTING BOXES, KNITTED UNION F...","(RED WOOLLY HOTTIE WHITE HEART., CREAM CUPID H...",0.080645,0.080645,0.080645,1.0,12.4,0.074142,inf
1649,(CREAM CUPID HEARTS COAT HANGER),"(RED WOOLLY HOTTIE WHITE HEART., WHITE HANGING...",0.080645,0.080645,0.080645,1.0,12.4,0.074142,inf
1650,(GLASS STAR FROSTED T-LIGHT HOLDER),"(RED WOOLLY HOTTIE WHITE HEART., CREAM CUPID H...",0.080645,0.080645,0.080645,1.0,12.4,0.074142,inf


In [32]:
rules['support'].value_counts()

0.080645    1382
Name: support, dtype: int64